This code uses attention mechanism to create an speech to text system

In [2]:
# this 2 line are my csv and audio feature extracted files.
# Path is csv file for audios and their texts
# PATH2 is audio feature extracted files. you'll create it in this code.
PATH  = '/kaggle/input/trsst/tr/'
PATH2 = '/kaggle/input/tr-attention-features/'

In [3]:
# you may need to install these libraries. use these commands to install them.
'''
all pip installs and commands
pip3 install pydub
pip3 install python_speech_features
apt-get install -y ffmpeg
'''

# ignore warnings.
import warnings
warnings.filterwarnings('ignore')

# Import libraries

In [ ]:
# You may see duplicated lines. don't matter

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import pydub
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile # reading the wavfile

# =========== < new import for approach 2 > ======================
from numpy.lib.stride_tricks import as_strided

from pydub import AudioSegment
from python_speech_features import mfcc
from mpl_toolkits.axes_grid1 import make_axes_locatable
import random
import tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Lambda)
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint   
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Activation
# =========== </new import for approach 2> ======================

import tensorflow as tf

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np
import os
import time
import json
from glob import glob
from PIL import Image
import pickle


# signal rate is a parameter for audios
SIGNAL_RATE = 16000
RNG_SEED = 123
NumberOfSamples = 1823 # write number of samples here. number of data you have.

# A little work with csv file

In [ ]:
def drop_col(df, cols):
    for col in cols:
        df.drop([col], axis=1, inplace=True)


def prepare_dataset(df_address):
    df =pd.read_csv(df_address, delimiter='\t', encoding='utf-8')
    drop_col(df, ['age', 'up_votes', 'client_id', 'accent', 'down_votes', 'gender'])
    # If you need duration of each file just uncomment next lines
#     duration_list = []
#     for path in list(df.path):
#         y, sr = librosa.load(PATH + 'clips/' +  path, sr=16000) # sound is 
#         duration = librosa.core.get_duration(y=y , sr=sr )
#         duration_list.append(duration)
#     df['durations'] = duration_list
    return df

In [ ]:
# you may change these lines
#prepare validation and train data just for now
partition='train'
train_attention = prepare_dataset(PATH+ 'train' + '.tsv' )

partition = 'dev'
dev_attention = prepare_dataset(PATH+ 'dev' + '.tsv')

you may be curious my csv file columns. We used Deep Speech Mozilla data. so Our csvs have these columns : 
*  age
* up_votes
* client_id
* accent
* down_votes
* gender
* sentece
* path

I just used 2 last columns

# Useful funcitons to Preprocess Audio data

In [ ]:
# this is the way to pad all the y of audios to same size.
def padarray(A, size):
    t = size - len(A)
    return np.pad(A, pad_width=(0, t), mode='constant')

def spectrogram(samples, fft_length=256, sample_rate=2, hop_length=128):
    """
    Compute the spectrogram for a real signal.
    The parameters follow the naming convention of
    matplotlib.mlab.specgram
    Args:
        samples (1D array): input audio signal
        fft_length (int): number of elements in fft window
        sample_rate (scalar): sample rate
        hop_length (int): hop length (relative offset between neighboring
            fft windows).
    Returns:
        x (2D array): spectrogram [frequency x time]
        freq (1D array): frequency of each row in x
    Note:
        This is a truncating computation e.g. if fft_length=10,
        hop_length=5 and the signal has 23 elements, then the
        last 3 elements will be truncated.
    """
    assert not np.iscomplexobj(samples), "Must not pass in complex numbers"

    window = np.hanning(fft_length)[:, None]
    window_norm = np.sum(window**2)  

    # The scaling below follows the convention of
    # matplotlib.mlab.specgram which is the same as
    # matlabs specgram.
    scale = window_norm * sample_rate

    trunc = (len(samples) - fft_length) % hop_length
    x = samples[:len(samples) - trunc]

    # "stride trick" reshape to include overlap
    nshape = (fft_length, (len(x) - fft_length) // hop_length + 1)
    nstrides = (x.strides[0], x.strides[0] * hop_length)
    x = as_strided(x, shape=nshape, strides=nstrides)

    # window stride sanity check
    assert np.all(x[:, 1] == samples[hop_length:(hop_length + fft_length)])

    # broadcast window, compute fft over columns and square mod
    x = np.fft.rfft(x * window, axis=0)
    x = np.absolute(x)**2

    # scale, 2.0 for everything except dc and fft_length/2
    x[1:-1, :] *= (2.0 / scale)
    x[(0, -1), :] /= scale

    freqs = float(sample_rate) / fft_length * np.arange(x.shape[0])

    return x, freqs


def spectrogram_from_file(filename, step=10, window=20, max_freq=None,
                          eps=1e-14):
    """ Calculate the log of linear spectrogram from FFT energy
    Params:
        filename (str): Path to the audio file
        step (int): Step size in milliseconds between windows
        window (int): FFT window size in milliseconds
        max_freq (int): Only FFT bins corresponding to frequencies between
            [0, max_freq] are returned
        eps (float): Small value to ensure numerical stability (for ln(x))
    """
#     asssume we don't need next line ==>  PATH + 'clips/' + filename
    audio, sample_rate = librosa.load(PATH + 'clips/' + filename, sr=16000, duration = 10)
    audio = padarray(audio, 160000)
    if audio.ndim >= 2:
        audio = np.mean(audio, 1)
    if max_freq is None:
        max_freq = sample_rate / 2
    if max_freq > sample_rate / 2:
        raise ValueError("max_freq must not be greater than half of "
                            " sample rate")
    if step > window:
        raise ValueError("step size must not be greater than window size")
    hop_length = int(0.001 * step * sample_rate)
    fft_length = int(0.001 * window * sample_rate)
    pxx, freqs = spectrogram(
        audio, fft_length=fft_length, sample_rate=sample_rate,
        hop_length=hop_length)
#     print(pxx)
#     print(freqs)
    ind = np.where(freqs <= max_freq)[0][-1] + 1
    return np.transpose(np.log(pxx[:ind, :] + eps))

def calc_mfcc(filename):
    dim=26
    audio, sample_rate = librosa.load(PATH + 'clips/' + filename, sr=16000, duration = 10)
    audio = padarray(audio, 160000)
    return mfcc(audio, sample_rate, numcep=dim)

def load_audio(filename):
    audio, sample_rate = librosa.load(PATH + 'clips/' + filename, sr=16000, duration = 10)
    audio = padarray(audio, 160000)
    return audio, filename 
#     return np.zeros(160000), filename

# audios preprocessing

here I used MFCC feature extraction. mfcc dimension is 26. if you want to have 8000Hz you can use 13 as mfcc_dim

create a list of audio paths. then make loop and calculate mfcc or psetrogram of all audios. array result shape (No. audios, time_seq, feature)


In [ ]:
# create numpy file
train_features = [] # (No. audios, time_seq, feature)
for path in train_attention['path']:
    features_extracted.append(calc_mfcc(path))

with open('features_extracted_train_attention.npy', 'wb') as f:
    np.save(f, np.array(features_extracted))

# create numpy file
dev_features = [] # (No. audios, time_seq, feature)
for path in dev_attention['path']:
    features_extracted.append(calc_mfcc(path))

with open('features_extracted_dev_attention.npy', 'wb') as f:
    np.save(f, np.array(features_extracted))

use next cell if you created npy file and you just want to load it.

In [ ]:
# loaad numpy file
with open(PATH2 + 'features_extracted_train_attention.npy', 'rb') as f:
    train_features = np.load(f)
    
with open(PATH2 + 'features_extracted_dev_attention.npy', 'rb') as f:
    dev_features = np.load(f)

# captions Preprocessing and tokenizing the 
we have 2 approach :

1. tf.keras.preprocessng.text.Tokenizer
1. my own code ( we don't have it here. find this aproach in CTC_Custom_Generator folder)


In [ ]:
# Store captions and image names in vectors
all_captions = []
tokenized_captions = []
# for annot in train_attention['sentence']:
#     caption = '<start> ' + annot + ' <end>'
#     all_captions.append(caption)

tokenizer = tf.keras.preprocessing.text.Tokenizer( oov_token="<unk>", filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ', char_level=True, lower=True )
tokenizer.fit_on_texts( list(dev_attention['sentence']) + list(train_attention['sentence']) )
# tokenizer.fit_on_texts(train_attention['sentence'])
train_seqs = tokenizer.texts_to_sequences(train_attention['sentence'])
dev_seqs = tokenizer.texts_to_sequences(dev_attention['sentence'])

In [ ]:
print(f'sentence : {train_attention["sentence"][0]} , tokenized length : {len(train_seqs[0])}, original sentence length : {len(train_attention["sentence"][0])} ')

In [ ]:
tokenizer.word_index

In [ ]:
print(train_seqs[0])
print(dev_seqs[0])

In [ ]:
# add start and end to each sentence. add start and end to dict . add padding to dict and sentences. 
start_index_in_dict = len(tokenizer.word_index) + 1
end_index_in_dict = len(tokenizer.word_index) + 2

tokenizer.word_index['<start>'] = start_index_in_dict
tokenizer.index_word[start_index_in_dict] = '<start>'
tokenizer.word_index['<end>'] = end_index_in_dict
tokenizer.index_word[end_index_in_dict] = '<end>'
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'
def myfunc(x):
    x.insert(0,start_index_in_dict)
    x.insert(len(x),end_index_in_dict)
    return x

train_seqs = list(map(myfunc, train_seqs))
dev_seqs = list(map(myfunc, dev_seqs))

cap_vector_train = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')
cap_vector_dev = tf.keras.preprocessing.sequence.pad_sequences(dev_seqs, padding='post')

In [ ]:
print(cap_vector_train[0])
print(cap_vector_dev[0])
print(cap_vector_train.shape)

In [ ]:
tokenizer.word_index

# create tf.dataset

In [ ]:
train_features.shape

In [ ]:
# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 32
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = len(tokenizer.word_index) + 1
num_steps = len(train_features) // BATCH_SIZE
# Shape of the vector extracted from mfcc is (999, 26), for spectrogram is (999, 166)
# These two variables represent that vector shape
features_shape = 26 # 166 for spectrogram
attention_features_shape = 999

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((train_features, cap_vector_train))
# Shuffle and batch
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
# dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# attention encoder decoder

# Model
Fun fact: the decoder below is identical to the one in the example for Neural Machine Translation with Attention.

The model architecture is inspired by the Show, Attend and Tell paper.

* In this example, you extract the features from mfcc giving us a vector of shape (999, 26).
* This vector is then passed through the CNN Encoder (which consists of a single Fully connected layer).
* The RNN (here GRU) attends over the frequencies to predict the next word.

In [ ]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, features, hidden):
    # features(CNN_encoder output) shape == (batch_size, 999, embedding_dim => mand dadam (256) )

    # hidden shape == (batch_size, hidden_size).  hidden hidden state ghabli too balas.
    # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
    hidden_with_time_axis = tf.expand_dims(hidden, 1)

    # score shape == (batch_size, 999, hidden_size)
    score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

    # attention_weights shape == (batch_size, 999, 1)
    # you get 1 at the last axis because you are applying score to self.V
    attention_weights = tf.nn.softmax(self.V(score), axis=1)

    # context_vector shape after sum == (batch_size, hidden_size)
    context_vector = attention_weights * features
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [ ]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 999, embedding_dim (256))
        self.fc = tf.keras.layers.Dense(256)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [ ]:
class RNN_Decoder(tf.keras.Model):
  def __init__(self, embedding_dim, units, vocab_size):
    super(RNN_Decoder, self).__init__()
    self.units = units

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.units ,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc1 = tf.keras.layers.Dense(self.units)
    self.fc2 = tf.keras.layers.Dense(vocab_size)

    self.attention = BahdanauAttention(self.units)

  def call(self, x, features, hidden):
    # defining attention as a separate model
    
    context_vector, attention_weights = self.attention(features, hidden)
    
    x = tf.concat([tf.expand_dims(context_vector, 1), tf.expand_dims(x, 1)], axis=-1)
#     print(x.shape)
    
    # passing the concatenated vector to the GRU
    output, state = self.gru(x)

    # shape == (batch_size, max_length, hidden_size)
    x = self.fc1(output)

    # x shape == (batch_size * max_length, hidden_size)
    x = tf.reshape(x, (-1, x.shape[2]))

    # output shape == (batch_size * max_length, vocab)
    x = self.fc2(x)

    return x, state, attention_weights

  def reset_state(self, batch_size):
    return tf.zeros((batch_size, self.units))

In [ ]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [ ]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
  start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])

In [ ]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

@tf.function
def train_step(img_tensor, target):
  loss = 0

  # initializing the hidden state for each batch
  # because the captions are not related from audio to audio
  hidden = decoder.reset_state(batch_size=target.shape[0])
  current_batch_size = img_tensor.shape[0]
  dec_input = tf.expand_dims([float(tokenizer.word_index['<start>'])] * current_batch_size, 1)

  with tf.GradientTape() as tape:
      features = encoder(img_tensor)

      for i in range(1, target.shape[1]):
          # passing the features through the decoder
          predictions, hidden, _ = decoder(dec_input, features, hidden)

          loss += loss_function(target[:, i], predictions)

          # using teacher forcing
          dec_input = tf.expand_dims(float(target[:, i]), 1)
#           print(target[:,i])
#           print(predictions[:, i])
#           dec_input = predictions[:, i]
        

  total_loss = (loss / int(target.shape[1]))

  trainable_variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, trainable_variables)

  optimizer.apply_gradients(zip(gradients, trainable_variables))

  return loss, total_loss

In [ ]:
EPOCHS = 20

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
      ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))